<a href="https://colab.research.google.com/github/Samir-atra/CancerDetector/blob/main/Meningioma_Detector/Cancer_Detector_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Imports

!pip install keras_tuner -q
!pip install wandb -qU
import tensorflow as tf
import keras_tuner
import numpy as np
import os
import pathlib
import matplotlib.pyplot as plt
import datetime
import tensorboard
import IPython
import sklearn
import cv2
import sys
import time
import wandb
from wandb.keras import WandbMetricsLogger
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [11]:
# Mounting google drive for training in google co lab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# WandB

wandb.init(
    project="cancer_detector",
    config={
        "epochs": 10,
        "batch_size": 5,
        "lr": 0.0005670323757088328
        })

config = wandb.config

In [13]:
# Dataset loading

data_path = pathlib.Path('/content/drive/MyDrive/archiveX3/Training/')

data_path_test = pathlib.Path('/content/drive/MyDrive/archiveX3/Testing/')
AUTOTUNE=tf.data.AUTOTUNE

start = time.time()
dataset_path = tf.keras.utils.image_dataset_from_directory(        # Training dataset
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='training',
    seed= 1,
    batch_size=config.batch_size,
    image_size=(299, 299),
    color_mode="rgb",
    shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

dataset_path_val = tf.keras.utils.image_dataset_from_directory(      #Validation dataset
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='validation',
    seed= 2,
    batch_size=config.batch_size,
    image_size=(299, 299),
    color_mode="rgb",
    shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

# dataset_path_test = tf.keras.utils.image_dataset_from_directory(      # Testing dataset
#     data_path_test,
#     labels= 'inferred',
#     seed= 3,
#     batch_size=5,
#     image_size=(299, 299),
#     color_mode="rgb",
#     shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

end = time.time()
print("time elapsed = ", (end - start))


Found 4096 files belonging to 2 classes.
Using 3277 files for training.
Found 4096 files belonging to 2 classes.
Using 819 files for validation.
time elapsed =  3.7418885231018066


In [14]:
# catch and prefetch for shorter training time

# AUTOTUNE=tf.data.AUTOTUNE
# dataset_path = dataset_path.cache().prefetch(buffer_size=AUTOTUNE)
# dataset_path_val = dataset_path_val.cache().prefetch(buffer_size=AUTOTUNE)

In [15]:
# downloading the weights oft the base model

base_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape = (299, 299, 3),
    include_top = False,
    weights = "imagenet"
)

In [16]:
#saving the downloaded base_model

saving_path = pathlib.Path('/content/drive/MyDrive/archiveX3/Training/SavedBaseModel.h5')

base_model.save(saving_path)

In [ ]:
#Model

model_path = pathlib.Path('/content/drive/MyDrive/archiveX3/Training/SavedBaseModel.h5')

base_model = tf.keras.models.load_model(model_path)                 #Loading base_model

base_model.trainable = False                                        # Setting the model as non-trainable

rescaling = tf.keras.Sequential([
  tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)             # Rescaling to (1, -1) range required for inceptionV3 model
])
augmentation = tf.keras.Sequential([                                # Applying augmentations o the images
   tf.keras.layers.RandomFlip("horizontal"),
   tf.keras.layers.RandomRotation(0.1)
])

# def build_model(hp):

inputs = tf.keras.Input(shape=(299, 299, 3))
x = augmentation(inputs)
x = rescaling(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)                   # "softmax" in the final layer for decision making
model = tf.keras.Model(inputs, outputs)

# learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-3, sampling="log")

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = config.lr),        # compiling with low learning rate
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly = True)

#  return model

# build_model(keras_tuner.HyperParameters())

# tuner = keras_tuner.RandomSearch(
#     hypermodel=build_model,
#     objective="val_accuracy",
#     max_trials=3,
#     executions_per_trial=2,
#     overwrite=True,
#     directory="/content/drive/MyDrive/Transfer",
#     project_name="Tuner",
# )

# tuner.search_space_summary()

# tuner.search(dataset_path, epochs=5, validation_data = dataset_path_val)

# tuner.results_summary()


g = model.fit(                                                               # fitting the whole model for non-trainable base
    dataset_path,
    epochs=config.epochs,
    validation_data = dataset_path_val,
    callbacks = [WandbMetricsLogger(log_freq=10)])
    #verbose = 1)
# for item in g.history.items():
#     for j in range(config.epochs):
#         metrics = {"train/train_loss": item[1][j]}
#         wandb.log(metrics)


# model.evaluate(dataset_path_test, batch_size=5, verbose=1)               # evaluating using the test dataset

model.summary()
wandb.finish()

# base_model.trainable = True                                           # switching the base_model to trainable

# for layer in base_model.layers:                                       # Swtching all batchnormalization layer of the base_model to non-trainable to not lose weights
#   if isinstance(layer, tf.keras.layers.BatchNormalization):
#     layer.trainable = False

# model.compile(
#     optimizer = tf.keras.optimizers.Adam(learning_rate = 0.000001),    # compiling the model with a trainable base_model
#     loss=tf.losses.SparseCategoricalCrossentropy(),
#     metrics=['accuracy'],
#     run_eagerly = True)

# # model.fit(                                                            # fitting for another 10 epochs starting where the non-trainable ended
#     dataset_path,
#     epochs=30,
#     initial_epoch=15,
#     validation_data = dataset_path_val)
    #verbose = 1)

# model.evaluate(dataset_path_test, batch_size=5, verbose=1)

# model.summary()

# saving_path = pathlib.Path('/content/drive/MyDrive/archiveX3/SavedModelWithFT/')

# tf.keras.models.save_model(model,                                     # saving the fully trained model
#                            saving_path,
#                            overwrite=True,
#                            save_format='tf'
#                            )

Epoch 1/10
656/656 [==============================] - 271s 412ms/step - loss: 0.2462 - accuracy: 0.9017 - val_loss: 0.1821 - val_accuracy: 0.9341
Epoch 2/10
656/656 [==============================] - 267s 407ms/step - loss: 0.1612 - accuracy: 0.9408 - val_loss: 0.2477 - val_accuracy: 0.9182
Epoch 3/10
656/656 [==============================] - 267s 407ms/step - loss: 0.1295 - accuracy: 0.9530 - val_loss: 0.1454 - val_accuracy: 0.9475
Epoch 4/10
656/656 [==============================] - 268s 409ms/step - loss: 0.1157 - accuracy: 0.9594 - val_loss: 0.1346 - val_accuracy: 0.9487
Epoch 5/10
656/656 [==============================] - 268s 409ms/step - loss: 0.1105 - accuracy: 0.9643 - val_loss: 0.1279 - val_accuracy: 0.9487
Epoch 6/10
656/656 [==============================] - 267s 406ms/step - loss: 0.0972 - accuracy: 0.9646 - val_loss: 0.1234 - val_accuracy: 0.9536
Epoch 7/10
127/656 [====>.........................] - ETA: 3:07 - loss: 0.0996 - accuracy: 0.9654

In [ ]:
# for _ in range(5):
#     # 🐝 initialise a wandb run
#     wandb.init(
#         project="cancer_detector",
#         config={
#             "epochs": 10,
#             "batch_size": 5,
#             "lr": 1e-3
#             })

    # Copy your config
    # config = wandb.config


    # Dataset_path
    # Dataset_path_val
    # model